# 1. 确定版本

In [1]:
import sys

print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 22:22:05) [MSC v.1916 64 bit (AMD64)]


In [5]:
type(sys.version)

str

In [6]:
type(sys.version_info)

sys.version_info

# 2. 遵循PEP 8风格指南

这里列举几条我需要注意的

## 采用内联的否定词

In [7]:
a = 1
b = 2

In [9]:
if a is not 2:
    print("TRUE")
else:
    print("FALSE")

TRUE


In [10]:
# 也能运行，但不要这样写，用上面的方式
if not a is 2:
    print("TRUE")
else:
    print("FALSE")

TRUE


## 不要使用检测长度的方式来判断列表、字符串是否为空

In [13]:
c = []
d = ""

In [15]:
# 默认这些空的为FALSE
if not c:
    print("TRUE")
else:
    print("FALSE")
if not d:
    print("TRUE")
else:
    print("FALSE")

TRUE
TRUE


In [14]:
# 不要用以下的方式
if len(c) == 0:
    print("TRUE")
else:
    print("FALSE")
if len(d) == 0:
    print("TRUE")
else:
    print("FALSE")

TRUE
TRUE


类似的，检测非空的时候，也不要用`len`，而是直接使用列表或字符串即可，因为非空的对象默认被当做`True`。

# 3. 了解bytes、str和unicode的区别

* python3包含两种描述字符序列的类：bytes和str。其中前者的实例包含的是原始的8位值，后者的实例包含的是Unicode字符。（在python2中，是str和unicode两个类）。
* bytes类有`decode`方法，str类有`encode`方法，这两个方法实现了两个类的互相转换。
* bytes实例中储存着编码使用的类型，比如utf-8，所以我们我们直接打印bytes实例的时候，实际上是运行了`__repr__`方法，其会再将此实例按照编码类型转换成对应的字符打印，只是前面会加个记号`b`；而str类可以看做一种特殊的编码类型，所有其他的编码类型都可以与之进行相互转换，所以其可以看做是一种“桥梁”编码类型。
* 编写程序的时候，先解码成Unicode字符（使用bytes的`decode`方法），然后进行程序中的操作，然后再编码成对应的字符类型（使用str的`encode`方法），然后保存到文件中进行保存或传输，即编码和解码的工作要放在最外层做，这样便于我们更换各种不同的编码形式。（一般来时，这个用于保存和传输的编码类型是utf-8）

In [16]:
a = "student"

In [29]:
aa = a.encode("utf-8")

In [34]:
print(aa.__doc__)

bytes(iterable_of_ints) -> bytes
bytes(string, encoding[, errors]) -> bytes
bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
bytes(int) -> bytes object of size given by the parameter initialized with null bytes
bytes() -> empty bytes object

Construct an immutable array of bytes from:
  - an iterable yielding integers in range(256)
  - a text string encoded using the specified encoding
  - any object implementing the buffer API.
  - an integer


In [35]:
print(a.__doc__)

str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


一般来说，我们需要编写两个辅助函数：

第一个，不管接受的是str还是bytes(这里默认是utf-8)，返回的都是str的方法：

In [36]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode("utf-8")
    else:
        value = bytes_or_str
    return value

第二个，不管接受的是str还是bytes，返回的都是bytes的方法：

In [37]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode("utf-8")
    else:
        value = bytes_or_str
    return value

在python3中，以上的内容会影响到文件读写的操作，比如使用以下操作：

In [38]:
import os

a = os.urandom(10)
print(type(a))

with open("./tmp/random.bin", "w") as f:
    f.write(a)

<class 'bytes'>


TypeError: write() argument must be str, not bytes

这是因为python3为了操作的方便，把encode成utf-8和写入文件这两个操作组合在了一起，使用参数`encoding`来控制。所以`write`接受的是str实例，而非bytes实例。如果想要接受bytes实例，则需要使用`"wb"`模式。

In [40]:
with open("./tmp/random.bin", "wb") as f:
    f.write(a)

或者先给decode成str实例，当然这种操作是浪费了的。

在读取的时候是类似的，为了方便python3将读取和decode整合到了一起，如果不想要decode过程，使用`"rb"`模式。

# 4. 使用辅助函数来取代复杂的表达式

如果有一行非常长的表达式，难以理解（即使使用了`if else`表达式），则建议单独使用一个辅助函数来进行封装使用。

# 5. 了解切割序列的方法

* 就是切片操作，这种操作只要对象实现了`__getitem__`和`__setitem__`方法就可以使用。
* 即使索引越界也不会出现问题（但这在索引访问单个元素的时候会error）。
* 切片操作会创建新的对象，但如果切片作为左值来操作则会修改列表中的内容。
* `b = a[:]`会得到原对象的一个拷贝。

In [43]:
# 这里介绍一个奇怪的结果，即当使用负值来进行切片操作时

a = list(range(10))
print(a)
print(a[-5:])
print(a[-0:]) # 如果按照负值切片的逻辑，应该返回的是[]，但这里实际上是先把-0=0，然后再进行的切片

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [44]:
b = a[:]
assert a == b
assert b is a

AssertionError: 

赋值不会改变对象（内存地址）。

In [46]:
b = a
a[:3] = [100, 101, 102]
assert a is b

# 6. 单次切片操作不要同时指定start、end和stride

In [47]:
a

[100, 101, 102, 3, 4, 5, 6, 7, 8, 9]

In [48]:
a[1:5:2]

[101, 3]

stride的一个常用操作是进行列表反转：

In [49]:
w = "abcdefg"
w[::-1]

'gfedcba'

这个甚至在其bytes（编码是ASCII）形式下也是有用的：

In [50]:
w = b"abcdef"
w[::-1]

b'fedcba'

但当我们涉及到utf-8编码的时候就不一定了

In [55]:
# 如果全是英文，则utf-8编码和ASCII一致
w = "abcdefg"
print(w.encode("utf-8"))
print(w.encode("utf-8")[::-1])
print(w.encode("utf-8")[::-1].decode("utf-8"))
# 如果还有其他类型的字符，比如中文，则就不行了
w = "谢谢!"
print(w[::-1])
print(w.encode("utf-8"))
print(w.encode("utf-8")[::-1])
print(w.encode("utf-8")[::-1].decode("utf-8"))

b'abcdefg'
b'gfedcba'
gfedcba
!谢谢
b'\xe8\xb0\xa2\xe8\xb0\xa2!'
b'!\xa2\xb0\xe8\xa2\xb0\xe8'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa2 in position 1: invalid start byte

我们可以看出，这个排列反转是以一字节为单位进行的。但utf-8编码汉字是以3个字节为一个汉字的，所以会报错。但在str实例上是可以的，其本质是以一个字符一个字符为单位储存的。

**尽量不要把stride和start、end写在一起，如果非要用，请保证其是正的。可以考虑分成两步，先做切片再做步进。**

In [56]:
b = a[::2]
c = b[1:-1]

但这样做会产生一次浅拷贝，如果程序对内存用量要求比较高，则可以考虑使用`itertools.islide`方法，其不允许为start、end、stride使用负值。

# 7. 使用列表推导来取代map和filter

因为列表推导比map和filter形成的代码更加清晰。

# 8. 不要使用含有两个以上表达式的列表推导

In [57]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
flat

[1, 2, 3, 4, 5, 6, 7, 8, 9]

现在我们来实现：得到其中被3所整除、且所在行的各元素之和大于等于10的单元格：

In [58]:
[[x for x in row if x % 3 == 0] for row in matrix if sum(row) >= 10]

[[6], [9]]

但这比较难以理解，使用`if else`来编写可能更好。

# 9. 使用生成器表达式来改写数据量较大的列表推导

就是把`[]`改成`()`，这样返回的是一个iterator。然后再使用`next`逐渐返回值。

有意思的是，我们可以编写嵌套的生成器，使用一个`next`来实现两个生成器的同时生成。

In [60]:
a = list(range(100))
b = ((i, i**0.5) for i in a)
b

<generator object <genexpr> at 0x0000016D1B3AF5E8>

In [61]:
next(b)

(0, 0.0)

# 10. 使用enumerate代替range

`enumerate(list)`来代替`range(len(list))`

而且`enumerate`提供了第二个参数来决定开始计数时用的值（默认是0）

In [62]:
flavor_list = ["vanilla", "chocolate", "pecan", "strawberry"]
for i, flavor in enumerate(flavor_list):
    print("%d: %s" % (i, flavor))

0: vanilla
1: chocolate
2: pecan
3: strawberry


In [63]:
for i, flavor in enumerate(flavor_list, 2):
    print("%d: %s" % (i, flavor))

2: vanilla
3: chocolate
4: pecan
5: strawberry


# 11. 使用zip来同时遍历两个（及以上）迭代器

* python2中的`zip`处理大列表不好，请使用`itertools.izip`。
* 如果两个列表长度不同，则自动到最短的列表耗尽未知，如果想要到长的列表，请使用`itertools.zip_longest`。

# 12. 不要在for和while后面写else块

* `for`循环中没有遇到`break`，则执行后面的`else`。
* 最好不要写。

In [65]:
for i in range(5):
    if i > 5:
        break
else:
    print("TRUE")

TRUE


In [66]:
for i in range(5):
    if i > 3:
        break
else:
    print("TRUE")

# 13. 合理利用try/except/else/finally结构中的每个代码块

* finally用来处理一些清理工作，比如关闭文件句柄。
* else用来处理异常没有发生时的处理过程。